#Installing all packages



###Important to run the cell twice, then click on "Restart runtime" button in output and run once again.


In [1]:
!pip install --upgrade pip
!pip install deeppavlov
!python -m deeppavlov install squad_bert
!pip uninstall tensorflow
!pip uninstall tensorflow-gpu
!pip install tensorflow-gpu==1.15.2 
!pip install transformers==2.8.0
import tensorflow as tf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-06-15 19:00:24.666 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'squad_bert' as '/usr/local/lib/python3.7/dist-packages/deeppavlov/configs/squad/squad_bert.json'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /tmp/pip-req-build-d_m2118j
  Running command git clone --filter=blob:none --quiet https://github.com/deepmipt/bert.git /tmp/pip-req-build-d_m2118j
  Resolved https://github.com/deepmipt/bert.git to commit 741d9bed9d52c6a9409ca27d8bf284615645618b
  Preparing metadata (setup.py) ... done
2022-06-15 19:00:31.713 WARNING in 'deeppavlov.utils.pip_wrapper.pip_wrapper'['pip_wrapper'] at line 34: found tensorflow-gpu installed, so upg

In [2]:

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

from deeppavlov.core.data.utils import download
from deeppavlov.dataset_readers.basic_classification_reader import BasicClassificationDatasetReader
from deeppavlov.dataset_iterators.basic_classification_iterator import BasicClassificationDatasetIterator
from deeppavlov.models.preprocessors.bert_preprocessor import BertPreprocessor
from deeppavlov.models.preprocessors.torch_transformers_preprocessor import TorchTransformersPreprocessor
from deeppavlov.core.data.simple_vocab import SimpleVocabulary
from deeppavlov.models.preprocessors.one_hotter import OneHotter
from deeppavlov.models.classifiers.proba2labels import Proba2Labels
from deeppavlov.models.bert.bert_classifier import BertClassifierModel
from deeppavlov.metrics.accuracy import sets_accuracy

from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt


Num GPUs Available:  1


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.


##Dataset preparation


In [3]:
#1 getting a dataset together; .csvs are uploaded manually
foul = pd.read_csv("foul.csv")
nd = pd.read_csv("neutral_desc.csv")
neutral = nd[nd["toxic"]==0]
dataset = pd.concat([foul, neutral], ignore_index=True)

In [4]:
#2 cleaning text part of the dataset
def cleanup(text):
    text = text.lower()
    regular = r'[\*+\#+\№\"\-+\+\=+\?+\&\^\.+\;\,+\>+\(\)\/+\:\\+]'
    text = re.sub(regular, '', text)
    return text

text2 = []
for text in dataset['comment']: 
    text = cleanup(text)
    text2.append(text)
dataset['descr'] = text2

In [10]:
#3 splitting data into train-val-test
xx = dataset.descr.values
yy = dataset.toxic.values

x_train, x_test, y_train, y_test = train_test_split(xx, yy, test_size=0.15, shuffle = True)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.20, shuffle = True)

x_train = pd.DataFrame(data = x_train, columns=['text'])
y_train = pd.DataFrame(data = y_train, columns=['tag'])
train = pd.concat([x_train, y_train], axis=1)

train.to_csv("train.csv")
train_path = "./train.csv"

x_val = pd.DataFrame(data = x_val, columns=['text'])
y_val = pd.DataFrame(data = y_val, columns=['tag'])
val = pd.concat([x_val, y_val], axis=1)

val.to_csv("val.csv")
val_path = "./val.csv"

x_test = pd.DataFrame(data = x_test, columns=['text'])
y_test = pd.DataFrame(data = y_test, columns=['tag'])
test = pd.concat([x_test, y_test], axis=1)

test.to_csv("test.csv")
test_path = "./test.csv"

print(len(train), len(val), len(test))
print(len(train) + len(val) + len(test))

14652 3663 3233
21548


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
#things done manually now:
# - "data" folder is created
# - train-val-test .csv files are put there 
# - three BERT tar.gz files are uploaded

reader = BasicClassificationDatasetReader()
data = reader.read(data_path="./data", 
                   train="train.csv", valid="val.csv",test = 'test.csv' ,
                   x="text", y="tag")

!tar -xf '/content/drive/MyDrive/mds22/ru_conversational_cased_L-12_H-768_A-12.tar.gz'
#!tar -xf '/content/drive/MyDrive/MDS 2022/rubert_cased_L-12_H-768_A-12_v2.tar.gz'
#!tar -xf '/content/drive/MyDrive/MDS 2022/sentence_ru_cased_L-12_H-768_A-12.tar.gz'




```
# This is formatted as code
```

##Training phase: ruConvBERT



In [12]:
#Defining helpful blocks

def roc_auc(y_test,y_pred_tag):
    y_pred_tag = [ y_p[1] for y_p in y_pred_tag ]
    roc_auc = roc_auc_score(y_test, y_pred_tag)
    return roc_auc

In [13]:
#model elements

iterator = BasicClassificationDatasetIterator(data, shuffle=True)

PRETR_BERT_PATH = "./ru_conversational_cased_L-12_H-768_A-12"

bert_preprocessor = BertPreprocessor(vocab_file=os.path.join(PRETR_BERT_PATH,"vocab.txt"),
                                     do_lower_case=False,
                                     max_seq_length=256)

input_features = bert_preprocessor(["мама мыла раму"])
vocab = SimpleVocabulary(save_path="./binary_classes.dict")
vocab.fit(xx)

#defining model
bert_classifier = BertClassifierModel(
    n_classes=2,
    return_probas=True,
    one_hot_labels=False,
    bert_config_file=os.path.join(PRETR_BERT_PATH,"bert_config.json"),
    pretrained_bert=os.path.join(PRETR_BERT_PATH,"bert_model.ckpt"),
    save_path="./model",
    load_path="./model",
    keep_prob=0.5,
    learning_rate=1e-05,
    learning_rate_drop_patience=5,
    learning_rate_drop_div=2.0)


2022-06-15 19:11:07.695 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 49: No load path is set for SimpleVocabulary in 'infer' mode. Using save path instead








The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.





Instructions for updating:
Use standard file APIs to check for files with this prefix.


2022-06-15 19:11:26.720 INFO in 'deeppavlov.models.bert.bert_classifier'['bert_classifier'] at line 99: [initializing model with Bert from /content/ru_conversational_cased_L-12_H-768_A-12/bert_model.ckpt]



INFO:tensorflow:Restoring parameters from /content/ru_conversational_cased_L-12_H-768_A-12/bert_model.ckpt


In [14]:
#Training phase

best_score = 0
patience = 3
impatience = 0
BATCH_SIZE = 16

DF_IN_USE = dataset
plot_every = int(len(dataset)/BATCH_SIZE)


for ep in range(30):
    #train  
    nbatches = 0
    train_loss = []

    for x, y in iterator.gen_batches(batch_size=BATCH_SIZE, 
                                     data_type="train", shuffle=True):
        x_feat = bert_preprocessor(x)
        loss_dict = bert_classifier.train_on_batch(x_feat, y)
        train_loss.append(loss_dict['loss'])
        nbatches += 1
        print('\r' + str(nbatches*BATCH_SIZE/len(dataset)), end='' )
    print('\n')

#val
    y_valid_trues = []
    y_valid_preds = []

    for x, y in iterator.gen_batches(batch_size=BATCH_SIZE, 
                                        data_type="valid", shuffle=True):
            y_valid_pred = bert_classifier(bert_preprocessor(x))
            y_valid_preds.extend(y_valid_pred)
            y_valid_trues.extend(y)
    
    score = roc_auc(y_valid_trues, y_valid_preds)

    y_valid_preds_class = np.argmax( np.array(y_valid_preds), axis=1 )
    y_valid = np.array([int(i) for i in y_valid_trues])

#score
    pr, rec, f, _ = precision_recall_fscore_support(y_valid, y_valid_preds_class, average='weighted')
    print("roc_auc", score)
    print("precision", pr)
    print("recall", rec)
    print("fscore_weighted", f)
    print("acc", accuracy_score(y_valid, y_valid_preds_class))

    print("Epochs done: {}. Valid roc_auc: {}".format(ep + 1, score))
    if score > best_score:
        bert_classifier.save()
        print("New best score. Saving model.")
        best_score = score    
        impatience = 0
    else:
        impatience += 1
        if impatience == patience:
            print("Out of patience. Stop training.")
            break


0.6801559309448673



2022-06-15 19:28:39.388 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 75: [saving model to /content/model]


roc_auc 0.9988091219271066
precision 0.990186478638567
recall 0.9901719901719902
fscore_weighted 0.9901684774010349
acc 0.9901719901719902
Epochs done: 1. Valid roc_auc: 0.9988091219271066
New best score. Saving model.
0.6801559309448673



2022-06-15 19:43:50.908 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 75: [saving model to /content/model]


roc_auc 0.9993661309574046
precision 0.9872130616584823
recall 0.9871689871689872
fscore_weighted 0.9871743196804774
acc 0.9871689871689872
Epochs done: 2. Valid roc_auc: 0.9993661309574046
New best score. Saving model.
0.6801559309448673

roc_auc 0.9987652665083251
precision 0.9907665567642725
recall 0.9907179907179907
fscore_weighted 0.9907123466522223
acc 0.9907179907179907
Epochs done: 3. Valid roc_auc: 0.9987652665083251
0.6801559309448673

roc_auc 0.9981234100526868
precision 0.9874718022616445
recall 0.9874419874419874
fscore_weighted 0.9874461974250468
acc 0.9874419874419874
Epochs done: 4. Valid roc_auc: 0.9981234100526868
0.6801559309448673

roc_auc 0.9988406194615579
precision 0.9885421049164075
recall 0.9885339885339886
fscore_weighted 0.988530591035884
acc 0.9885339885339886
Epochs done: 5. Valid roc_auc: 0.9988406194615579
Out of patience. Stop training.


## Training phase: ruBERT


In [ ]:
#model elements


PRETR_BERT_PATH = "./rubert_cased_L-12_H-768_A-12_v2"

bert_preprocessor = BertPreprocessor(vocab_file=os.path.join(PRETR_BERT_PATH,"vocab.txt"),
                                     do_lower_case=False,
                                     max_seq_length=256)

#defining model
bert_classifier = BertClassifierModel(
    n_classes=2,
    return_probas=True,
    one_hot_labels=False,
    bert_config_file=os.path.join(PRETR_BERT_PATH,"bert_config.json"),
    pretrained_bert=os.path.join(PRETR_BERT_PATH,"bert_model.ckpt"),
    save_path="./model",
    load_path="./model",
    keep_prob=0.5,
    learning_rate=1e-05,
    learning_rate_drop_patience=5,
    learning_rate_drop_div=2.0)

#Training phase

best_score = 0
patience = 3
impatience = 0
BATCH_SIZE = 16

DF_IN_USE = dataset
plot_every = int(len(dataset)/BATCH_SIZE)


for ep in range(30):
    #train  
    nbatches = 0
    train_loss = []

    for x, y in iterator.gen_batches(batch_size=BATCH_SIZE, 
                                     data_type="train", shuffle=True):
        x_feat = bert_preprocessor(x)
        loss_dict = bert_classifier.train_on_batch(x_feat, y)
        train_loss.append(loss_dict['loss'])
        nbatches += 1
        print('\r' + str(nbatches*BATCH_SIZE/len(dataset)), end='' )
    print('\n')

#val
    y_valid_trues = []
    y_valid_preds = []

    for x, y in iterator.gen_batches(batch_size=BATCH_SIZE, 
                                        data_type="valid", shuffle=True):
            y_valid_pred = bert_classifier(bert_preprocessor(x))
            y_valid_preds.extend(y_valid_pred)
            y_valid_trues.extend(y)
    
    score = roc_auc(y_valid_trues, y_valid_preds)

    y_valid_preds_class = np.argmax( np.array(y_valid_preds), axis=1 )
    y_valid = np.array([int(i) for i in y_valid_trues])

#score
    pr, rec, f, _ = precision_recall_fscore_support(y_valid, y_valid_preds_class, average='weighted')
    print("roc_auc", score)
    print("precision", pr)
    print("recall", rec)
    print("fscore_weighted", f)
    print("acc", accuracy_score(y_valid, y_valid_preds_class))

    print("Epochs done: {}. Valid roc_auc: {}".format(ep + 1, score))
    if score > best_score:
        bert_classifier.save()
        print("New best score. Saving model.")
        best_score = score    
        impatience = 0
    else:
        impatience += 1
        if impatience == patience:
            print("Out of patience. Stop training.")
            break


2022-06-15 11:25:16.686 INFO in 'deeppavlov.models.bert.bert_classifier'['bert_classifier'] at line 99: [initializing model with Bert from /content/rubert_cased_L-12_H-768_A-12_v2/bert_model.ckpt]


INFO:tensorflow:Restoring parameters from /content/rubert_cased_L-12_H-768_A-12_v2/bert_model.ckpt
0.6801559309448673



2022-06-15 11:41:00.311 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 75: [saving model to /content/model]


roc_auc 0.9989970718132941
precision 0.9863517267665602
recall 0.9863499863499864
fscore_weighted 0.9863460715191523
acc 0.9863499863499864
Epochs done: 1. Valid roc_auc: 0.9989970718132941
New best score. Saving model.
0.6801559309448673



2022-06-15 11:56:35.379 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 75: [saving model to /content/model]


roc_auc 0.9991002431267604
precision 0.9837176883038591
recall 0.9836199836199836
fscore_weighted 0.9836015176924269
acc 0.9836199836199836
Epochs done: 2. Valid roc_auc: 0.9991002431267604
New best score. Saving model.
0.6801559309448673

roc_auc 0.9990653981897601
precision 0.9874419874419874
recall 0.9874419874419874
fscore_weighted 0.9874419874419874
acc 0.9874419874419874
Epochs done: 3. Valid roc_auc: 0.9990653981897601
0.6801559309448673

roc_auc 0.9989646508719112
precision 0.9863670915441017
recall 0.9863499863499864
fscore_weighted 0.9863537644942656
acc 0.9863499863499864
Epochs done: 4. Valid roc_auc: 0.9989646508719112
0.3066641915723037

KeyboardInterrupt: ignored

##Training phase: Sentence ruBERT

In [ ]:
#model elements


PRETR_BERT_PATH = "./sentence_ru_cased_L-12_H-768_A-12"

bert_preprocessor = BertPreprocessor(vocab_file=os.path.join(PRETR_BERT_PATH,"vocab.txt"),
                                     do_lower_case=False,
                                     max_seq_length=256)

#defining model
bert_classifier = BertClassifierModel(
    n_classes=2,
    return_probas=True,
    one_hot_labels=False,
    bert_config_file=os.path.join(PRETR_BERT_PATH,"bert_config.json"),
    pretrained_bert=os.path.join(PRETR_BERT_PATH,"bert_model.ckpt"),
    save_path="./model",
    load_path="./model",
    keep_prob=0.5,
    learning_rate=1e-05,
    learning_rate_drop_patience=5,
    learning_rate_drop_div=2.0)

#Training phase

best_score = 0
patience = 3
impatience = 0
BATCH_SIZE = 16

DF_IN_USE = dataset
plot_every = int(len(dataset)/BATCH_SIZE)


for ep in range(30):
    #train  
    nbatches = 0
    train_loss = []

    for x, y in iterator.gen_batches(batch_size=BATCH_SIZE, 
                                     data_type="train", shuffle=True):
        x_feat = bert_preprocessor(x)
        loss_dict = bert_classifier.train_on_batch(x_feat, y)
        train_loss.append(loss_dict['loss'])
        nbatches += 1
        print('\r' + str(nbatches*BATCH_SIZE/len(dataset)), end='' )
    print('\n')

#val
    y_valid_trues = []
    y_valid_preds = []

    for x, y in iterator.gen_batches(batch_size=BATCH_SIZE, 
                                        data_type="valid", shuffle=True):
            y_valid_pred = bert_classifier(bert_preprocessor(x))
            y_valid_preds.extend(y_valid_pred)
            y_valid_trues.extend(y)
    
    score = roc_auc(y_valid_trues, y_valid_preds)

    y_valid_preds_class = np.argmax( np.array(y_valid_preds), axis=1 )
    y_valid = np.array([int(i) for i in y_valid_trues])

#score
    pr, rec, f, _ = precision_recall_fscore_support(y_valid, y_valid_preds_class, average='weighted')
    print("roc_auc", score)
    print("precision", pr)
    print("recall", rec)
    print("fscore_weighted", f)
    print("acc", accuracy_score(y_valid, y_valid_preds_class))

    print("Epochs done: {}. Valid roc_auc: {}".format(ep + 1, score))
    if score > best_score:
        bert_classifier.save()
        print("New best score. Saving model.")
        best_score = score    
        impatience = 0
    else:
        impatience += 1
        if impatience == patience:
            print("Out of patience. Stop training.")
            break


2022-06-15 12:36:18.542 INFO in 'deeppavlov.models.bert.bert_classifier'['bert_classifier'] at line 99: [initializing model with Bert from /content/sentence_ru_cased_L-12_H-768_A-12/bert_model.ckpt]


INFO:tensorflow:Restoring parameters from /content/sentence_ru_cased_L-12_H-768_A-12/bert_model.ckpt
0.6801559309448673



2022-06-15 12:52:01.858 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 75: [saving model to /content/model]


roc_auc 0.9992734073136796
precision 0.9872389063727125
recall 0.9871689871689872
fscore_weighted 0.9871570592405526
acc 0.9871689871689872
Epochs done: 1. Valid roc_auc: 0.9992734073136796
New best score. Saving model.
0.6801559309448673

roc_auc 0.9991938699574832
precision 0.9885407232762149
recall 0.9885339885339886
fscore_weighted 0.9885298599874709
acc 0.9885339885339886
Epochs done: 2. Valid roc_auc: 0.9991938699574832
0.6801559309448673



2022-06-15 13:23:08.342 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 75: [saving model to /content/model]


roc_auc 0.9992749223109404
precision 0.9869120111764246
recall 0.9868959868959869
fscore_weighted 0.9868893236673711
acc 0.9868959868959869
Epochs done: 3. Valid roc_auc: 0.9992749223109404
New best score. Saving model.
0.6801559309448673

roc_auc 0.9992075049328312
precision 0.9871771913056512
recall 0.9871689871689872
fscore_weighted 0.9871712218652453
acc 0.9871689871689872
Epochs done: 4. Valid roc_auc: 0.9992075049328312
0.6801559309448673

roc_auc 0.9989879818297288
precision 0.9882624500444407
recall 0.9882609882609883
fscore_weighted 0.9882580487894633
acc 0.9882609882609883
Epochs done: 5. Valid roc_auc: 0.9989879818297288
0.24874698347874513